# **Guessing pizza price prediction with topping,diameter.....etc.**

<img style="{margin-left:50px}" src="https://c.tenor.com/XMeGq6VyxQoAAAAC/friends-joey-tribbiani.gif">

# DATAFLOW:
1. Data Collection
2. Data cleaning and checking
3. Visualize data and checking
4. spliting the data
5. best-fit model with pycaret 😊
6. evaluation metric

# Importing the required packages

In [ ]:
!pip install pycaret

In [ ]:
import numpy as np
import pandas as pd
import category_encoders as ce
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from pycaret.regression import *
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as MSE
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.inspection import permutation_importance
import catboost as cb


<h4>Reading the data from the csv file using pandas inbulit package for handling csv datastructures✌️ </h4>

In [ ]:
df=pd.read_csv('../input/pizza-price-prediction/pizza_v1.csv')

In [ ]:
df.head()

<h4>Total number of rows in the dataset</h4>

In [ ]:
# Number of rows in dataset 
df.size

<h3>No null values😋</h3>

In [ ]:
# checking for null values
df.isnull().sum()

<h3>Splitting into categorical and numerical columns</h3>

In [ ]:
# splitting into categorical and numerical columns
df_numerical=df.select_dtypes(exclude=['object'])
df_categorical=df.select_dtypes(include=['object'])

In [ ]:
df.dtypes

In [ ]:
# cleaning the categorical columns
df['company'].value_counts()
df["price_rupiah"].value_counts()

<h3>Renaming the columns</h3>

In [ ]:
df.rename(columns = {'price_rupiah':'Cost','company':'Company','diameter':'Diameter','variant':'Variant','size':'Size','extra_sauce':'Extra_sauce','extra_cheese':'Extra_cheese'}, inplace = True)

In [ ]:
df.columns

<H3>Data Visualization<?H3>

In [ ]:
plt.figure(figsize=(10,8))
plt.pie(df['Company'].value_counts(),autopct='%.1f',labels=df['Company'].unique())
plt.title('Companies Weigtage')
plt.show()

In [ ]:
sns.boxplot(x="Company", y="Diameter", data=df)
plt.show()

In [ ]:
sns.boxplot(x="topping", y="Diameter", data=df)
plt.xticks(rotation =90)
plt.show()

In [ ]:
df_company=df.groupby('Company').agg({'Company':['count']})
sns.kdeplot(df['Company'].value_counts(), color='r', shade=True)

In [ ]:
df_company=df.groupby('topping').agg({'topping':['count']})
sns.kdeplot(df['topping'].value_counts(), color='r', shade=True)

In [ ]:
sns.histplot(data=df, x="Company",hue='Extra_cheese')

In [ ]:
sns.histplot(data=df, x="Company",hue='Extra_sauce')

In [ ]:
#encodded columns
encoding_columns=['Company','topping','Variant','Size','Extra_sauce','Extra_cheese']

<h3>Our machine learning model can understand only numbers, So we should convert the categorical columns into numerical .So that
    we are able to come with a better fit of the data</h3>
<img src='https://media3.giphy.com/media/26TFT7az4DofCxqShL/giphy.gif?cid=ecf05e47ubqnhg3cwc6uao5nr6rqa1y6x6jun0a11yfuzlle&rid=giphy.gif&ct=g'>

    

<h3>Important custom function for data cleaning </h3>

<h4>Refining cost is used to replace some reduntant symbols in the "Cost" column</h4>

In [ ]:
# cost cleaning
def refining_cost(col,df):
    df[col]=df[col].map(lambda x:x.replace('Rp',''))
    df[col]=df[col].map(lambda x:x.replace(',',''))
    

<h4>categorical_encoding is used to encode the categorical values into numerical values</h4>

In [ ]:
def categorical_encoding(value,df):
    one_hot_encoder=ce.OneHotEncoder(cols=value,return_df=True,use_cat_names=True)
    df_final = one_hot_encoder.fit_transform(df)
    return df_final
        
       
    
        

In [ ]:
df=categorical_encoding("Company",df)
df=categorical_encoding("topping",df)
df=categorical_encoding('Variant',df)
df=categorical_encoding('Size',df)
df=categorical_encoding("Extra_sauce",df)
df=categorical_encoding('Extra_cheese',df)


In [ ]:
refining_cost('Cost',df)

In [ ]:
df['Cost']=df['Cost'].astype('float64')

In [ ]:
df.dtypes

In [ ]:
X=df.drop(['Cost'],axis=1)
Y=df['Cost']

In [ ]:
print(X.shape)

<h3>Splitting Our data into train and test </h3>

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=1372)

In [ ]:
X_train, X_test, y_train, y_test=np.array(X_train), np.array(X_test), np.array(y_train), np.array(y_test)

<h3>Scaling the data to get a more genralized model,which could allocate proper weightage to each
    and every features in training set</h3>
<img src='https://media.giphy.com/media/3o7TKx9IHKtbtSo3Mk/giphy.gif'>

In [ ]:
scaler = MinMaxScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)


In [ ]:
scaler = MinMaxScaler()
y_train=scaler.fit_transform(y_train.reshape(-1,1))
y_test=scaler.transform(y_test.reshape(-1,1))

In [ ]:
y_test

<h4>PyCaret is an open-source, low-code machine learning library in Python that aims to reduce the cycle time from hypothesis to insights. It is well suited for seasoned data scientists who want to increase the productivity of their ML experiments by using PyCaret in their workflows or for citizen data scientists and those new to data science with little or no background in coding.</h4>

In [ ]:

clf = setup(df, target = 'Cost',normalize=True)

In [ ]:
df

<h3>Pycaret will be able to come up with a efficient models</h3>

<p>we have to choose the model with highest coefficient of dtermination</p>

In [ ]:
compare_models()

In [ ]:
model = create_model('catboost')

<H3>RMSE is calculated for evaluvating for the cat boost regressor</H3>

In [ ]:
train_dataset = cb.Pool(X_train, y_train) 
test_dataset = cb.Pool(X_test, y_test)
model = cb.CatBoostRegressor(loss_function="RMSE")
grid = {'iterations': [100, 150, 200],
        'learning_rate': [0.03, 0.1],
        'depth': [2, 4, 6, 8],
        'l2_leaf_reg': [0.2, 0.5, 1, 3]}
model.grid_search(grid, train_dataset)
pred = model.predict(X_test)
rmse = (np.sqrt(mean_squared_error(y_test, pred)))
r2 = r2_score(y_test, pred)






In [ ]:
print("RMSE: {:.2f}".format(rmse))
print("R2: {:.2f}".format(r2))